In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.request import urlretrieve
from IPython.core.display import display, HTML
%matplotlib inline

In [2]:
pd.options.display.max_columns = None
display(HTML("<style>.container { width:100% !important; }</style>"))

# Docker

## A Very Good Walkthrough
* Part 1: [The conceptual landscape of Docker](https://towardsdatascience.com/learn-enough-docker-to-be-useful-b7ba70caeb)
* Part 2: [Useful Docker Terms](https://towardsdatascience.com/learn-enough-docker-to-be-useful-1c40ea269fa8)
* Part 3: [Dockerfile Writing](https://towardsdatascience.com/learn-enough-docker-to-be-useful-b0b44222eef5)
* Part 4: [Slim down Docker images](https://towardsdatascience.com/slimming-down-your-docker-images-275f0ca9337e)
* Part 5: [Vital Docker Commands](https://towardsdatascience.com/15-docker-commands-you-should-know-970ea5203421)
* Part 6: [Docker Volumes](https://towardsdatascience.com/pump-up-the-volumes-data-in-docker-a21950a8cd8)
* Part 7: [Docker Security](https://towardsdatascience.com/top-20-docker-security-tips-81c41dd06f57)

```docker
How to make docker secret compatible with this docker-compose?

hi,

I am trying to use docker secret with a fusionauth image. But it seem like the "/run/secrets" part is parsed as a string. I tried to add _FILE in the back of password, but it did not work.

Does anyone know how I can make this compose file docker-secret compatible?

version: '3.3'

services:
  fusionauth:
    image: fusionauth/fusionauth-app:latest
    secrets:
      - source: test_pw
        target: postgres_password
    environment:
      POSTGRES_PASSWORD: /run/secrets/postgres_password
      DATABASE_URL: jdbc:postgresql://db:5432/fusionauth
      SEARCH_TYPE: database
      FUSIONAUTH_APP_URL: http://fusionauth:9011
    networks:
     - db
    restart: unless-stopped
    ports:
      - 9011:9011
    volumes:
      - fa_config:/usr/local/fusionauth/config

networks:
  db:
    driver: bridge

volumes:
  db_data:
  fa_config:

secrets:
  test_pw:
    external: true

```


The challenge is that the image and software in it has to support reading the password from a file. Some images don't do that out of the box, and only support reading from an environment variable. To make it work, you might have to build a new image with an updated endpoint/run command or something. I am not really sure what you need to do for fusionauth.

Looking at the Dockerfile, it seems like you could add an endpoint that would read the secret and set environment variable, then exec the run command.

    https://github.com/FusionAuth/fusionauth-containers/tree/master/docker/fusionauth/fusionauth-app
    https://fusionauth.io/docs/v1/tech/reference/configuration/
        See the Lookup process section.




There are two things going on here. Your question is about how to user docker-compose with secrets and the docker-compose file that you have in your OP is mainly correct. So all is good there. That's how you use secrets with docker-compose.

The problem you're having is that you can't get your secrets into your fusionauth container. The documentation on docker hub for fusionauth/fusionauth-app is scarce and doesn't say much about the environment variables.

Secrets in docker aren't really magical. It's just a convenience that image maintainers give to users of their image to use the docker secrets feature. For example, take a look at the PostgreSQL image on docker hub, their documentation says, "Currently, [secrets are] only supported for POSTGRES_INITDB_ARGS, POSTGRES_PASSWORD, POSTGRES_USER, and POSTGRES_DB.".

How the Postgres gets the secrets into the environment variables is through the startup script. If you take a look at the PostgreSQL start up script that's ran when you start a Postgres container, the start up script is just looking for environment variables named <env_var>_FILE, reading the contents of that file, and setting the environment using the contents of the file.

So for an environment variable like POSTGRES_PASSWORD, the script looks for any environment variables named POSTGRES_PASSWORD_FILE, and if it exists, it will read the file and set POSTGRES_PASSWORD to the contents of that file.

So in order to get this to work in your situation, you need to be familiar with the fusionauth image. If there's an entrypoint script in that container, then you can modify the entrypoint script to read the POSTGRES_PASSWORD_FILE environment variable, and set POSTGRES_PASSWORD there.

The code for reading <env_var>_FILE will look something like this (I took this straight from the Postgres docker-entrypoint script):

```bash
file_env() {
    local var="$1"
    local fileVar="${var}_FILE"
    local def="${2:-}"
    if [ "${!var:-}" ] && [ "${!fileVar:-}" ]; then
            echo >&2 "error: both $var and $fileVar are set (but are exclusive)"
            exit 1
    fi
    local val="$def"
    if [ "${!var:-}" ]; then
            val="${!var}"
    elif [ "${!fileVar:-}" ]; then
            val="$(< "${!fileVar}")"
    fi
    export "$var"="$val"
    unset "$fileVar"
}

# Call the function on to do a replacement for POSTGRES_PASSWORD
file_env 'POSTGRES_PASSWORD'
```

If there is no entrypoint script that you can overwrite, then you can build your own Dockerfile using fusionauth as a base image and then set your own Entrypoint to be a script that does this. Best of luck.
